In [1]:
import torch
import intel_extension_for_pytorch as ipex
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


class Classifier(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=2),
            nn.ReLU(),
            nn.Conv2d(8, 8*8, 3, stride=2),
            nn.ReLU(),
            nn.AvgPool2d(3, 2),
            nn.Flatten(),
            nn.Dropout(),
            nn.Linear(256, 128), 
            nn.Dropout(),
            nn.Linear(128, 10), 
            nn.Softmax()
        )

    def forward(self, x):
        return self.model(x)
    

def train(dataloader, model, loss_fn, optimizer, dtype=None):
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):
        if dtype is not None:
            X, y = X.to(dtype), y.to(dtype)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn, backend="cuda", dtype=None):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            if dtype is not None:
                X, y = X.to(dtype), y.to(dtype)
            X, y = X.to(backend), y.to(backend)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


/home/arthur/.local/lib/python3.10/site-packages/intel_extension_for_pytorch/llm/__init__.py:9: UserWarning: failed to use huggingface generation fuctions due to: No module named 'transformers'.
  warnings.warn(f"failed to use huggingface generation fuctions due to: {e}.")


In [2]:
training_data = datasets.FashionMNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor(),
    )

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
############## import ipex ###############
##########################################

model = Classifier()
model.eval()

bs = 96
train_dataloader = DataLoader(training_data, batch_size=bs)


In [4]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

model.train()
dtype = torch.float8_e4m3fn
# from intel_extension_for_pytorch._isa_help import Float8Format
# dtype = Float8Format.kFloat8_E4M3
model, optimizer = ipex.optimize(model, optimizer=optimizer, dtype=dtype)

print(model.parameters().__next__().dtype)

for i in range(20):
    train(train_dataloader, model, loss_fn, optimizer, dtype=None)

/home/arthur/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


torch.float32
loss: 2.302077  [   96/60000]
loss: 2.302940  [ 9696/60000]
loss: 2.301334  [19296/60000]
loss: 2.301256  [28896/60000]
loss: 2.302502  [38496/60000]
loss: 2.303185  [48096/60000]
loss: 2.302539  [57696/60000]
loss: 2.301679  [   96/60000]
loss: 2.302111  [ 9696/60000]
loss: 2.301704  [19296/60000]
loss: 2.300772  [28896/60000]
loss: 2.303035  [38496/60000]
loss: 2.302470  [48096/60000]
loss: 2.304465  [57696/60000]
loss: 2.301762  [   96/60000]
loss: 2.302619  [ 9696/60000]
loss: 2.302622  [19296/60000]
loss: 2.302905  [28896/60000]
loss: 2.302669  [38496/60000]
loss: 2.302727  [48096/60000]
loss: 2.302892  [57696/60000]
loss: 2.302049  [   96/60000]
loss: 2.302384  [ 9696/60000]
loss: 2.301648  [19296/60000]
loss: 2.301941  [28896/60000]
loss: 2.302754  [38496/60000]
loss: 2.303049  [48096/60000]
loss: 2.303161  [57696/60000]
loss: 2.301345  [   96/60000]
loss: 2.302206  [ 9696/60000]
loss: 2.301672  [19296/60000]
loss: 2.302043  [28896/60000]
loss: 2.301762  [38496/600

In [5]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
test_dataloader = DataLoader(test_data, batch_size=bs)

test(test_dataloader, model, loss_fn, backend="cpu", dtype=None)

/home/arthur/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Test Error: 
 Accuracy: 8.4%, Avg loss: 2.300713 



In [6]:
print(model.parameters().__next__().dtype)

torch.float32
